In [ ]:
!rm -r /content/logs/*

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/scalars

In [ ]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing import image

(x_train, y_train), (x_validation, y_validation) = datasets.cifar100.load_data()

print(x_train.shape)
print(x_validation.shape)
print(y_train.shape)
print(y_validation.shape)

In [ ]:
from tensorflow.keras.applications.vgg19 import preprocess_input

# preprocess function will normalize (mean + standard deviation) but not scale the images
x_train = preprocess_input(x_train)
x_validation = preprocess_input(x_validation)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=100)
y_validation = to_categorical(y_validation, num_classes=100)

In [ ]:
import pickle
import requests
import tarfile

# download and extract files from archive
url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
response = requests.get(url, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|gz")
file.extractall(path=".")

with open('./cifar-100-python/meta', 'rb') as f:
    labels = pickle.load(f)['fine_label_names']

print(labels)

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

vgg19 = VGG19(include_top=False)
print('VGG layers:')
print(vgg19.summary())

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D

def load_VGG(transfer_strategy=None):

    if transfer_strategy is None:
        base_model = VGG19(weights=None, include_top=False, input_shape=(32,32,3))
    else:
        base_model = VGG19(weights='imagenet', include_top=False, input_shape=(32,32,3))

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='softmax'))

    if not (transfer_strategy is None) and transfer_strategy != 'imagenet':
        trainable = False

        # iterate over layers taken from VGG19 (= model.layers[0])
        for layer in model.layers[0].layers:
            if transfer_strategy in layer.name and not trainable:
                print('start unfreezing from layer {}'.format(layer.name))
                trainable = True

            # if freeze is False, then set layer to trainable
            layer.trainable = trainable

    return model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

# 3 transfer learning strategy are applied:
#   - no transfer at all (train from scratch)
#   - use ImageNet pre-trained weights and re-train all the model
#   - use ImageNet pre-trained weights and re-train only layers from block 4

for transfer_strategy in [None, 'imagenet', 'block5']:

    tensorboard_callback = TensorBoard(log_dir="logs/scalars/{}".format(transfer_strategy))

    checkpointer = ModelCheckpoint(filepath='model_{}.keras'.format(transfer_strategy), monitor='val_loss', verbose=1, save_best_only=True)

    model = load_VGG(transfer_strategy)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_validation, y_validation), batch_size=16, epochs=10, callbacks=[tensorboard_callback, checkpointer])